In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

### 데이터 읽어오기

In [ ]:
def extract(df):
    X = df.drop(columns=['MEDV'])
    Y = df['MEDV']
    return X, Y

# 데이터 읽기
data = pd.read_csv('data/train.csv').set_index('ID')
test = pd.read_csv('data/test.csv').set_index('ID')
input_x, input_y = extract(data)
submit_x = test

### 훈련 및 예측

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# 훈련용 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(input_x, input_y, test_size=0.2, random_state=100)
 
# 훈련
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 성능 계산
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:", round(rmse, 4))

### 결과 파일 생성

In [ ]:
# 결과 파일 생성
submit_y = pd.DataFrame(model.predict(submit_x), index=submit_x.index, columns=['MEDV'])
submit_y.to_csv("submission.csv", index=True)



### 결과 분석

In [ ]:
# 피처 이름과 가중치를 DataFrame으로 정리
weights_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Weight': model.coef_
})
weights_df = weights_df.sort_values(by='Weight', ascending=False)  # 가중치 크기순 정렬

print(weights_df)